# 3.   MODEL TRAINING

Import necessary libraries

In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import xgboost as xgb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

Set default Seaborn style

In [4]:
sns.set(style="whitegrid")

Load the training and test datasets

In [5]:
train_data_path = '../data/CMaps/train_FD001.txt'
test_data_path = '../data/CMaps/test_FD001.txt'
rul_data_path = '../data/CMaps/RUL_FD001.txt'

Define column names based on the dataset's structure

In [6]:
col_names = [
    'engine_id', 'time_in_cycles', 
    'operational_setting_1', 'operational_setting_2', 'operational_setting_3'
] + [f'sensor_{i}' for i in range(1, 27)]  # This creates sensor_1 to sensor_21

Load the train and test datasets

In [7]:
train_df = pd.read_csv(train_data_path, sep=' ', header=None, names=col_names)
test_df = pd.read_csv(test_data_path, sep=' ', header=None, names=col_names)

Drop columns that are completely empty (if any)

In [8]:
train_df.dropna(axis=1, how='all', inplace=True)
test_df.dropna(axis=1, how='all', inplace=True)

Load the Remaining Useful Life (RUL) data for the test set

In [10]:
rul_df = pd.read_csv(rul_data_path, header=None, names=['RUL'])

Add RUL values to the test dataframe
Since the RUL values correspond to each engine in the test set, we add them directly to the test_df

In [13]:
test_df['RUL'] = test_df['engine_id'].map(lambda x: rul_df.loc[x - 1, 'RUL'])
train_df['RUL'] = train_df['engine_id'].map(lambda x: rul_df.loc[x - 1, 'RUL'])

We will use train_df (training data) and test_df (test data)

In [14]:
# Define the features (X) and the target variable (y)
X = train_df.drop(columns=['RUL', 'engine_id', 'time_in_cycles'])  # Drop target and non-feature columns
y = train_df['RUL']  # Target variable

Split the training data into training and validation sets

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model 1: Random Forest Regressor
Initialize the model

In [16]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

Train the model

In [17]:
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Predict on the validation set

In [18]:
rf_predictions = rf_model.predict(X_val)

Evaluate the model

In [19]:
rf_mae = mean_absolute_error(y_val, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_predictions))

print(f"Random Forest - MAE: {rf_mae}")
print(f"Random Forest - RMSE: {rf_rmse}")

Random Forest - MAE: 35.718526774897015
Random Forest - RMSE: 41.19672796130431


## Model 2: XGBoost Regressor
Prepare data for XGBoost

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

Set the parameters for XGBoost

In [21]:
params = {'objective': 'reg:squarederror', 'eval_metric': 'rmse'}
xgb_model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')])

[0]	eval-rmse:41.40210
[1]	eval-rmse:41.16507
[2]	eval-rmse:41.04990
[3]	eval-rmse:41.01928
[4]	eval-rmse:40.95461
[5]	eval-rmse:40.98197
[6]	eval-rmse:40.98020
[7]	eval-rmse:41.01046
[8]	eval-rmse:41.00811
[9]	eval-rmse:41.02626
[10]	eval-rmse:41.04439
[11]	eval-rmse:41.04784
[12]	eval-rmse:41.10469
[13]	eval-rmse:41.12859
[14]	eval-rmse:41.14795
[15]	eval-rmse:41.16565
[16]	eval-rmse:41.17014
[17]	eval-rmse:41.23583
[18]	eval-rmse:41.25310
[19]	eval-rmse:41.25674
[20]	eval-rmse:41.28022
[21]	eval-rmse:41.31827
[22]	eval-rmse:41.34931
[23]	eval-rmse:41.39355
[24]	eval-rmse:41.41230
[25]	eval-rmse:41.45475
[26]	eval-rmse:41.44801
[27]	eval-rmse:41.45946
[28]	eval-rmse:41.51802
[29]	eval-rmse:41.54504
[30]	eval-rmse:41.56192
[31]	eval-rmse:41.58399
[32]	eval-rmse:41.61107
[33]	eval-rmse:41.65905
[34]	eval-rmse:41.67317
[35]	eval-rmse:41.67754
[36]	eval-rmse:41.73702
[37]	eval-rmse:41.75877
[38]	eval-rmse:41.77945
[39]	eval-rmse:41.79709
[40]	eval-rmse:41.83025
[41]	eval-rmse:41.84325
[4

Predict with XGBoost

In [22]:
xgb_predictions = xgb_model.predict(dval)

Evaluate the model

In [23]:
xgb_mae = mean_absolute_error(y_val, xgb_predictions)
xgb_rmse = np.sqrt(mean_squared_error(y_val, xgb_predictions))

print(f"XGBoost - MAE: {xgb_mae}")
print(f"XGBoost - RMSE: {xgb_rmse}")

XGBoost - MAE: 36.57939910888672
XGBoost - RMSE: 42.793807221646425


## Model 3: LSTM (Deep Learning)
Prepare data for LSTM model (reshape to [samples, time steps, features])

In [24]:
# LSTM requires 3D input, so reshape the data
X_train_lstm = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val_lstm = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))

Build the LSTM model

In [25]:
lstm_model = Sequential()
lstm_model.add(LSTM(units=64, return_sequences=False, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))  # Output layer for regression task

Compile the LSTM model

In [26]:
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

Train the LSTM model

In [27]:
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_data=(X_val_lstm, y_val))

Epoch 1/10
516/516 [==============================] - 6s 6ms/step - loss: 6935.1582 - val_loss: 6411.1675
Epoch 2/10
516/516 [==============================] - 3s 5ms/step - loss: 6146.7441 - val_loss: 5674.8774
Epoch 3/10
516/516 [==============================] - 2s 3ms/step - loss: 5443.8135 - val_loss: 5022.4893
Epoch 4/10
516/516 [==============================] - 2s 3ms/step - loss: 4826.5664 - val_loss: 4447.6294
Epoch 5/10
516/516 [==============================] - 2s 4ms/step - loss: 4286.0454 - val_loss: 3942.9131
Epoch 6/10
516/516 [==============================] - 2s 3ms/step - loss: 3802.7590 - val_loss: 3504.4429
Epoch 7/10
516/516 [==============================] - 2s 4ms/step - loss: 3393.5088 - val_loss: 3126.7161
Epoch 8/10
516/516 [==============================] - 2s 3ms/step - loss: 3050.6738 - val_loss: 2804.4951
Epoch 9/10
516/516 [==============================] - 2s 4ms/step - loss: 2743.9053 - val_loss: 2536.3176
Epoch 10/10
516/516 [=========================

Predict with LSTM

In [28]:
lstm_predictions = lstm_model.predict(X_val_lstm)

129/129 [==============================] - 1s 3ms/step


Evaluate the LSTM model

In [29]:
lstm_mae = mean_absolute_error(y_val, lstm_predictions)
lstm_rmse = np.sqrt(mean_squared_error(y_val, lstm_predictions))

print(f"LSTM - MAE: {lstm_mae}")
print(f"LSTM - RMSE: {lstm_rmse}")

LSTM - MAE: 41.97737503051758
LSTM - RMSE: 48.1351678584964
